In [1]:
using Plots
using Random
using Distributions

# 強度 ν_1 を計算する関数
function intensity_ν1(ν0_1, n_11, n_12, τ_1, times_1, times_2, t)
    z_1 = sum([(1/τ_1) * exp(-(t - ti)/τ_1) for ti in times_1])
    z_2 = sum([(1/τ_1) * exp(-(t - tj)/τ_1) for tj in times_2])
    return ν0_1 + n_11 * z_1 + n_12 * z_2
end

# 強度 ν_2 を計算する関数
function intensity_ν2(ν0_2, n_21, n_22, τ_2, times_1, times_2, t)
    z_1 = sum([(1/τ_2) * exp(-(t - ti)/τ_2) for ti in times_1])
    z_2 = sum([(1/τ_2) * exp(-(t - tj)/τ_2) for tj in times_2])
    return ν0_2 + n_21 * z_1 + n_22 * z_2
end

# 二次元ホークス過程をシミュレートする関数
function simulate_two_dimensional_hawkes_process(ν0_1, ν0_2, n_11, n_12, n_21, n_22, τ_1, τ_2, T)
    times_1 = Float64[]
    times_2 = Float64[]
    intensities_1 = Float64[]
    intensities_2 = Float64[]

    t = 0.0

    while t < T
        ν_1 = intensity_ν1(ν0_1, n_11, n_12, τ_1, times_1, times_2, t)
        ν_2 = intensity_ν2(ν0_2, n_21, n_22, τ_2, times_1, times_2, t)
        total_intensity = ν_1 + ν_2
        E = rand(Exponential(1))
        
        if total_intensity > 0
            dt = E / total_intensity
        else
            dt = T
        end

        t += dt
        if t < T
            if rand() < ν_1 / total_intensity  # 第1次元のイベントかどうか判定
                push!(times_1, t)
                push!(intensities_1, ν_1)
            else  # 第2次元のイベント
                push!(times_2, t)
                push!(intensities_2, ν_2)
            end
        end
    end

    return times_1, intensities_1, times_2, intensities_2
end

# パラメータ設定
ν0_1 = 0.01
ν0_2 = 0.2
n_11 = 0.495
n_12 = 0.495
n_21 = 0.495
n_22 = 0.495
τ_1 = 1.0
τ_2 = 1.5
T = 10000

# シミュレーション実行
times_1, intensities_1, times_2, intensities_2 = simulate_two_dimensional_hawkes_process(ν0_1, ν0_2, n_11, n_12, n_21, n_22, τ_1, τ_2, T)

# プロット
p1 = plot(times_1, intensities_1, label="Intensity 1", xlabel="Time", ylabel="Intensity", legend=:topright)
p2 = plot(times_2, intensities_2, label="Intensity 2", xlabel="Time", ylabel="Intensity", legend=:topright)

# 強度のヒストグラムを正規化してプロット
p3 = histogram(intensities_1, bins=50, label="Intensity 1 Distribution", xlabel="Intensity", ylabel="Density", normalize=true)
p4 = histogram(intensities_2, bins=50, label="Intensity 2 Distribution", xlabel="Intensity", ylabel="Density", normalize=true)

# 全てのプロットを表示
plot(p1, p2, p3, p4, layout=(2,2))


LoadError: InterruptException: